In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import warnings
from sklearn import preprocessing
warnings.filterwarnings("ignore")
np.random.seed(42)

In [2]:
df = pd.read_csv('train.csv')
y = df.target

df.drop(['ID', 'target'], axis=1, inplace=True)
df.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,...,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,NaN,NaN,C,NaN,9.191265,NaN,NaN,2.301630,NaN,1.312910,...,NaN,NaN,0.598896,AF,NaN,NaN,1.957825,0,NaN,NaN
2,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,...,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,6.542669,...,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,1.050328,...,NaN,NaN,NaN,Z,NaN,NaN,NaN,0,NaN,NaN


In [3]:
string_type = []
for column in df.columns:
    if type(df[column].values[0]) == str:
        string_type.append(column)
string_type.append('v113')
        
df[string_type] = df[string_type].fillna('zero')

df.fillna(-9999, inplace=True)
cat_features_ids = np.where(df.apply(pd.Series.nunique) < 10000)[0].tolist()

In [4]:
df.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,...,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,-9999.000000,-9999.000000,C,-9999.000000,9.191265,-9999.000000,-9999.000000,2.301630,-9999.000000,1.312910,...,-9999.000000,-9999.000000,0.598896,AF,-9999.000000,-9999.000000,1.957825,0,-9999.000000,-9999.000000
2,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,...,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,6.542669,...,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,-9999.000000,-9999.000000,C,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,1.050328,...,-9999.000000,-9999.000000,-9999.000000,Z,-9999.000000,-9999.000000,-9999.000000,0,-9999.000000,-9999.000000


In [5]:
le = preprocessing.LabelEncoder()
for column in string_type:
    df[column] = le.fit_transform(df[column])

In [6]:
df.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,1.335739,8.727474,2,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,...,8.000000,1.989780,0.035754,21,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,-9999.000000,-9999.000000,2,-9999.000000,9.191265,-9999.000000,-9999.000000,2.301630,-9999.000000,1.312910,...,-9999.000000,-9999.000000,0.598896,6,-9999.000000,-9999.000000,1.957825,0,-9999.000000,-9999.000000
2,0.943877,5.310079,2,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,...,9.333333,2.477596,0.013452,5,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,0.797415,8.304757,2,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,6.542669,...,7.018256,1.812795,0.002267,64,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,-9999.000000,-9999.000000,2,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,1.050328,...,-9999.000000,-9999.000000,-9999.000000,89,-9999.000000,-9999.000000,-9999.000000,0,-9999.000000,-9999.000000


In [6]:
train, test, y_train, y_test = train_test_split(df, y, test_size = 0.2)

In [7]:
lr = LogisticRegression()
lr.fit(train, y_train)
y_pred = lr.predict(test)
print('Accuracy score baseline:', accuracy_score(y_test, y_pred))

Accuracy score baseline: 0.7616881696916685


In [8]:
lgb_params = {
    'boosting_type': 'dart',
    'objective': 'binary',
    'metric':'binary_logloss',
    'learning_rate': 0.3,
    'max_bin':255,
    'num_leaves': 31,  
    'max_depth': -1,  
    'subsample': 0.7,  
    'colsample_bytree': 0.7,  
    'min_split_gain': 0,  
    'verbose': 0,
    'nthread': 32,
}

In [9]:
xgtrain = lgb.Dataset(train.values, y_train.values,categorical_feature = cat_features_ids)
xgvalid = lgb.Dataset(test.values,y_test.values,categorical_feature = cat_features_ids)

In [10]:
evals_results = {}
bst = lgb.train(lgb_params, 
                 xgtrain, 
                 valid_sets=[xgvalid], 
                 valid_names=['valid'], 
                 evals_result=evals_results, 
                 num_boost_round=10000000,
                 early_stopping_rounds=15,
                 verbose_eval=10)

Training until validation scores don't improve for 15 rounds.
[10]	valid's binary_logloss: 0.473342
[20]	valid's binary_logloss: 0.466094
[30]	valid's binary_logloss: 0.46422
[40]	valid's binary_logloss: 0.464024
[50]	valid's binary_logloss: 0.463845
[60]	valid's binary_logloss: 0.463892
[70]	valid's binary_logloss: 0.462854
[80]	valid's binary_logloss: 0.463078
[90]	valid's binary_logloss: 0.4627
[100]	valid's binary_logloss: 0.462505
[110]	valid's binary_logloss: 0.462739
[120]	valid's binary_logloss: 0.462452
Early stopping, best iteration is:
[111]	valid's binary_logloss: 0.462157


In [11]:
pred_lgb = bst.predict(test.values)
print('accuracy score with lgbm',accuracy_score(y_test,np.round(pred_lgb)))

accuracy score with lgbm 0.7850863765580581


In [12]:
clf = CatBoostClassifier(learning_rate=0.1, iterations=500, random_seed=42, logging_level='Silent')
clf.fit(train, y_train, cat_features=cat_features_ids)
pred_cbst = clf.predict_proba(test)[:,1]
print('accuracy score with Catboost',accuracy_score(y_test,np.round(pred_cbst)))

accuracy score with Catboost 0.7865733654056418


In [13]:
print('accuracy score with blend of lgb and cbst',accuracy_score(y_test,np.round(np.sqrt(pred_lgb*pred_cbst))))

accuracy score with blend of lgb and cbst 0.7884102339820687


In [14]:
print('accuracy score with blend of lgb and cbst',accuracy_score(y_test,np.round(0.5*pred_lgb + 0.5*pred_cbst)))

accuracy score with blend of lgb and cbst 0.788235294117647


In [21]:
original_score = 0.7865733654056418
best_score = 0.7884102339820687
improvement = np.abs(np.round(100*(original_score - best_score)/original_score,2))
print('overall improvement compare to non tuned model is {} %'.format(improvement))

overall improvement compare to non tuned model is 0.23 %


In [22]:
print('additional value is {} samples'.format(df.shape[0] * 0.23 / 100))

additional value is 262.9383 samples
